# Renewable energy sources optimization

### Problem formulation

We need to optimize the distribution of energy from multiple sources to different consumers to minimize the overall cost, while satisfying demand constraints.

### Parameters

- **Energy Sources**: Two sources (e.g., solar and wind), each with different capacities and costs.
- **Consumers**: Two consumers, each with a specific energy demand.
- **Objective**: Minimize the total cost of energy distribution while meeting all consumer demands without exceeding the source capacities.
- **Tools**: Linear programming, Quadratic Programming

- **Sources parameters**:

    | Source            | Capacity (kW) | Cost (€/kW) |
    |-------------------|---------------|-------------|
    | **I** (eg. Solar) | 100           | 0.15        |                    
    | **J**  (eg. Wind) | 150           | 0.20        |                   


- **Consumer Demands**:
    - Consumer A = 90 kW
    - Consumer B = 120 kW

### Formulation

- **Variables**:
    - $x_{i,a}$: Amount of energy transferred from source $i$ to consumer $a$.
- **Objective**
    We can define a linear cost, to be minimized:
    $$ \text{Cost} = cost_I \times (x_{I,A} + x_{I,B}) + cost_J \times(x_{J,A} + x_{J,B}) $$
- **Constraints**
  At any time these constraint must be met:
    1. Total energy supplied to each consumer meets their demand.
        - Demand of Consumer A: $x_{I,A} + x_{J,A} = 90$
        - Demand of Consumer B: $x_{I,B} + x_{J,B} = 120$
    2. The total energy taken from each source does not exceed its capacity.
        - Solar capacity: $x_{I,A} + x_{I,B} \leq 100$
        - Wind capacity: $x_{J,A} + x_{J,B} \leq 150$
    3. All energies must be positive
       - $x_{i,a} \geq 0$ for any $(i, a)$
         

In [1]:
# GLOBAL VARIABLES

In [2]:
from optimization.core.renewables import EnergySource, Consumer, EnergyDistribution

# Create EnergySource instances
solar = EnergySource("Solar", 100, 0.10)
wind = EnergySource("Wind", 150, 0.05)

# Create Consumer instances
consumer_a = Consumer("A", 90)
consumer_b = Consumer("B", 120)

# Create the energy distribution system
system = EnergyDistribution()

system.add_source(solar)
system.add_source(wind)

system.add_consumer(consumer_a)
system.add_consumer(consumer_b)

In [3]:
system.sources

{'Solar': {'capacity': 100, 'cost_per_unit': 0.1, 'unit': 'kW'},
 'Wind': {'capacity': 150, 'cost_per_unit': 0.05, 'unit': 'kW'}}

In [41]:
import numpy as np 

# cols: source, rows: customer

x_array = np.array([
    [30, 60], 
    [50, 70]
]) 

x_vector = x_array.flatten()

def cost_function_array(x_array):
    return solar.cost_per_unit * x_array[:, 0].sum() + wind.cost_per_unit * x_array[:, 1].sum()


def cost_function_vector(x_vector):
    return solar.cost_per_unit * (x_vector[0] + x_vector[2]) + wind.cost_per_unit * ( x_vector[1] + x_vector[3] )

cost_function_array(x_array) == cost_function_vector(x_vector)

True

In [68]:
xrange = np.linspace(1,120)
min_cost = 1e3
for x0 in xrange:
    for x1 in xrange:
        if x0 + x1 <= solar.capacity:
            for x2 in xrange:
                for x3 in xrange:
                    if x2 + x3 <= wind.capacity:
                        if (x0 + x2 - consumer_a.demand < 1e-3) & (x1 + x3 - consumer_b.demand < 1e-3):
                            cost = cost_function_vector([x0, x1, x2, x3])
                            if cost < min_cost:
                                min_cost = cost
                                print(min_cost)


0.30000000000000004


In [17]:
x_array[:, 0].sum() <= solar.capacity # sum first column, need to satisfy solar capacity <= 100
x_vector[::2].sum() <= solar.capacity

True

In [18]:
x_array[:, 1].sum() <= wind.capacity # sum first column, need to satisfy wind capacity <= 150
x_vector[1::2].sum() <= wind.capacity

True

In [20]:
x_array[0, :].sum() == consumer_a.demand # sum first row, need to satisfy Custumer's A need == 90
x_vector[:2].sum() == consumer_a.demand

True

In [21]:
x_array[1, :].sum() == consumer_b.demand # sum second row, need to satisfy Custumer's B need = 120
x_vector[2:].sum() == consumer_b.demand

True

In [56]:
from scipy.optimize import minimize

x = [0, 0, 0, 0]
res = minimize(
    lambda x: solar.cost_per_unit * (x[0] + x[2]) + wind.cost_per_unit * ( x[1] + x[3] ), #what we want to minimize
    x, 
    constraints = (
        {'type':'ineq','fun': lambda x: solar.capacity - (x[0] + x[2]) }, # sum first column, need to satisfy solar capacity <= 100
        {'type':'ineq','fun': lambda x: wind.capacity - (x[1] + x[3]) }, # sum first column, need to satisfy wind capacity <= 150
        {'type':'eq','fun': lambda x: x[0] + x[1] - consumer_a.demand }, # sum first row, need to satisfy Custumer's A need == 90
        {'type':'eq','fun': lambda x: x[2] + x[3] - consumer_b.demand } # sum second row, need to satisfy Custumer's B need = 120   
    ),
    bounds = ((0,None),(0,None),(0,None),(0,None)),
    method='SLSQP',options={'disp': True,'maxiter' : 100000})

print(res)
cost_function_vector(res.x)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 13.500000000000012
            Iterations: 8
            Function evaluations: 40
            Gradient evaluations: 8
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 13.500000000000012
       x: [ 2.250e+01  6.750e+01  3.750e+01  8.250e+01]
     nit: 8
     jac: [ 1.000e-01  5.000e-02  1.000e-01  5.000e-02]
    nfev: 40
    njev: 8


13.500000000000012

13.500000000000012

### TensorFlow Implementation

We will use TensorFlow to approximate a solution via a gradient descent approach, treating constraints as penalty terms in the loss function.

In [ ]:
import tensorflow as tf

# Initialize variables
x_solar_A = tf.Variable(50.0, trainable=True)  
x_solar_B = tf.Variable(50.0, trainable=True)
x_wind_A = tf.Variable(40.0, trainable=True)
x_wind_B = tf.Variable(80.0, trainable=True)

# Define constraints as penalties
def capacity_constraints():
    return [
        100 - (x_solar_A + x_solar_B),  # Solar capacity
        150 - (x_wind_A + x_wind_B),    # Wind capacity
        x_solar_A + x_wind_A - 90,      # Demand of Consumer A
        x_solar_B + x_wind_B - 120     # Demand of Consumer B
    ]

# Cost function
cost = 3*x_solar_A + 3*x_solar_B + 2*x_wind_A + 2*x_wind_B

# Optimizer
optimizer = tf.optimizers.SGD(learning_rate=0.01)

# Training step
def train_step():
    with tf.GradientTape() as tape:
        constraints = capacity_constraints()
        penalty = tf.reduce_sum(tf.square(constraints))  # Squared penalties for constraint violation
        loss = cost + penalty
    gradients = tape.gradient(loss, [x_solar_A, x_solar_B, x_wind_A, x_wind_B])
    optimizer.apply_gradients(zip(gradients, [x_solar_A, x_solar_B, x_wind_A, x_wind_B]))

# Optimization loop
for _ in range(1000):
    train_step()

# Output the results
print(f'x_solar_A: {x_solar_A.numpy()}, x_solar_B: {x_solar_B.numpy()}')
print(f'x_wind_A: {x_wind_A.numpy()}, x_wind_B: {x_wind_B.numpy()}')
